# Análisis de Datos de Netflix

Este notebook realiza un análisis exploratorio y limpieza de datos del dataset de Netflix.

**Dataset:** Netflix Shows Dataset de Kaggle
**Autor:** Osvaldo
**Fecha:** Agosto 2025

## 1. Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Librerías importadas exitosamente")

## 2. Carga y Exploración Inicial de Datos

In [ ]:
# Cargar el dataset
df = pd.read_csv('netflix_titles.csv')

print(f"Dataset cargado exitosamente")
print(f"Dimensiones del dataset: {df.shape}")
print(f"\nPrimeras 5 filas:")
df.head()

In [ ]:
# Información general del dataset
print("=== INFORMACIÓN GENERAL DEL DATASET ===")
print(f"Número de filas: {df.shape[0]:,}")
print(f"Número de columnas: {df.shape[1]}")
print(f"\nColumnas del dataset:")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

print("\n=== TIPOS DE DATOS ===")
df.info()

In [ ]:
# Análisis de valores faltantes
print("=== ANÁLISIS DE VALORES FALTANTES ===")
missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_df = pd.DataFrame({
    'Columna': missing_data.index,
    'Valores Faltantes': missing_data.values,
    'Porcentaje (%)': missing_percent.values
})

missing_df = missing_df[missing_df['Valores Faltantes'] > 0].sort_values('Valores Faltantes', ascending=False)
print(missing_df.to_string(index=False))

# Visualización de valores faltantes
plt.figure(figsize=(12, 6))
sns.heatmap(df.isnull(), cbar=True, cmap='viridis', yticklabels=False)
plt.title('Mapa de Valores Faltantes')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 3. Limpieza de Datos

In [ ]:
print("=== INICIANDO LIMPIEZA DE DATOS ===")

# Crear una copia del dataset original
df_clean = df.copy()
print(f"Dataset original: {df.shape[0]:,} filas")

# 1. Limpieza de la columna 'director'
df_clean['director'].fillna('Desconocido', inplace=True)

# 2. Limpieza de la columna 'cast'
df_clean['cast'].fillna('Sin información', inplace=True)

# 3. Limpieza de la columna 'country'
df_clean['country'].fillna('No especificado', inplace=True)

# 4. Limpieza de la columna 'date_added'
# Eliminar filas donde no hay fecha de agregado
df_clean = df_clean.dropna(subset=['date_added'])
print(f"Después de eliminar filas sin fecha: {df_clean.shape[0]:,} filas")

# 5. Convertir date_added a datetime
df_clean['date_added'] = pd.to_datetime(df_clean['date_added'], errors='coerce')

# 6. Limpieza de la columna 'rating'
df_clean['rating'].fillna('No clasificado', inplace=True)

# 7. Limpieza de la columna 'duration'
# Separar duration en número y unidad
df_clean['duration_value'] = df_clean['duration'].str.extract('(\d+)').astype(float)
df_clean['duration_unit'] = df_clean['duration'].str.extract('([a-zA-Z ]+)')

# 8. Crear columnas adicionales útiles
df_clean['year_added'] = df_clean['date_added'].dt.year
df_clean['month_added'] = df_clean['date_added'].dt.month
df_clean['day_of_week_added'] = df_clean['date_added'].dt.day_name()

# 9. Limpiar y separar géneros
df_clean['listed_in'] = df_clean['listed_in'].fillna('Sin categoría')

print(f"\n=== RESUMEN DE LIMPIEZA ===")
print(f"Filas originales: {df.shape[0]:,}")
print(f"Filas después de limpieza: {df_clean.shape[0]:,}")
print(f"Filas eliminadas: {df.shape[0] - df_clean.shape[0]:,}")

# Verificar valores faltantes después de la limpieza
print(f"\n=== VALORES FALTANTES DESPUÉS DE LIMPIEZA ===")
missing_after = df_clean.isnull().sum()
print(missing_after[missing_after > 0])

## 4. Análisis Exploratorio de Datos (EDA)

In [ ]:
# Distribución de tipos de contenido
print("=== DISTRIBUCIÓN DE TIPOS DE CONTENIDO ===")
type_counts = df_clean['type'].value_counts()
print(type_counts)
print(f"\nPorcentajes:")
print((type_counts / len(df_clean) * 100).round(2))

# Visualización
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de barras
type_counts.plot(kind='bar', ax=ax1, color=['#e74c3c', '#3498db'])
ax1.set_title('Distribución de Tipos de Contenido')
ax1.set_ylabel('Cantidad')
ax1.tick_params(axis='x', rotation=45)

# Gráfico de pastel
ax2.pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%', 
        colors=['#e74c3c', '#3498db'], startangle=90)
ax2.set_title('Proporción de Tipos de Contenido')

plt.tight_layout()
plt.show()

In [ ]:
# Análisis de contenido agregado por año
print("=== ANÁLISIS TEMPORAL ===")

# Contenido agregado por año
yearly_content = df_clean.groupby('year_added').size().reset_index(name='count')
yearly_by_type = df_clean.groupby(['year_added', 'type']).size().unstack(fill_value=0)

# Visualización
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12))

# Tendencia general
ax1.plot(yearly_content['year_added'], yearly_content['count'], 
         marker='o', linewidth=2, markersize=6, color='#2c3e50')
ax1.set_title('Contenido Agregado a Netflix por Año', fontsize=14, fontweight='bold')
ax1.set_xlabel('Año')
ax1.set_ylabel('Número de Títulos Agregados')
ax1.grid(True, alpha=0.3)

# Por tipo de contenido
yearly_by_type.plot(kind='bar', stacked=True, ax=ax2, 
                   color=['#e74c3c', '#3498db'], width=0.8)
ax2.set_title('Contenido Agregado por Año y Tipo', fontsize=14, fontweight='bold')
ax2.set_xlabel('Año')
ax2.set_ylabel('Número de Títulos')
ax2.legend(title='Tipo')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"\nAño con más contenido agregado: {yearly_content.loc[yearly_content['count'].idxmax(), 'year_added']}")
print(f"Cantidad: {yearly_content['count'].max():,} títulos")

In [ ]:
# Análisis de países
print("=== ANÁLISIS DE PAÍSES ===")

# Separar países (algunos títulos tienen múltiples países)
countries_split = df_clean['country'].str.split(', ').explode()
top_countries = countries_split.value_counts().head(15)

print("Top 15 países con más contenido:")
print(top_countries)

# Visualización
plt.figure(figsize=(12, 8))
top_countries.plot(kind='barh', color='steelblue')
plt.title('Top 15 Países con Más Contenido en Netflix', fontsize=14, fontweight='bold')
plt.xlabel('Número de Títulos')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Análisis de clasificaciones (ratings)
print("=== ANÁLISIS DE CLASIFICACIONES ===")

rating_counts = df_clean['rating'].value_counts()
print("Distribución de clasificaciones:")
print(rating_counts)

# Visualización
plt.figure(figsize=(12, 6))
rating_counts.plot(kind='bar', color='coral')
plt.title('Distribución de Clasificaciones de Contenido', fontsize=14, fontweight='bold')
plt.xlabel('Clasificación')
plt.ylabel('Número de Títulos')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Análisis por tipo de contenido
rating_by_type = pd.crosstab(df_clean['rating'], df_clean['type'])
print("\nClasificaciones por tipo de contenido:")
print(rating_by_type)

In [ ]:
# Análisis de duración
print("=== ANÁLISIS DE DURACIÓN ===")

# Separar por tipo para análisis de duración
movies_duration = df_clean[df_clean['type'] == 'Movie']['duration_value']
shows_duration = df_clean[df_clean['type'] == 'TV Show']['duration_value']

print(f"Películas - Duración promedio: {movies_duration.mean():.1f} minutos")
print(f"Películas - Duración mediana: {movies_duration.median():.1f} minutos")
print(f"Series TV - Temporadas promedio: {shows_duration.mean():.1f}")
print(f"Series TV - Temporadas mediana: {shows_duration.median():.1f}")

# Visualización
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Distribución de duración de películas
movies_duration.hist(bins=30, ax=ax1, color='lightblue', edgecolor='black', alpha=0.7)
ax1.axvline(movies_duration.mean(), color='red', linestyle='--', 
            label=f'Promedio: {movies_duration.mean():.1f} min')
ax1.axvline(movies_duration.median(), color='orange', linestyle='--', 
            label=f'Mediana: {movies_duration.median():.1f} min')
ax1.set_title('Distribución de Duración de Películas')
ax1.set_xlabel('Duración (minutos)')
ax1.set_ylabel('Frecuencia')
ax1.legend()

# Distribución de temporadas de series
shows_duration.hist(bins=15, ax=ax2, color='lightgreen', edgecolor='black', alpha=0.7)
ax2.axvline(shows_duration.mean(), color='red', linestyle='--', 
            label=f'Promedio: {shows_duration.mean():.1f} temp')
ax2.axvline(shows_duration.median(), color='orange', linestyle='--', 
            label=f'Mediana: {shows_duration.median():.1f} temp')
ax2.set_title('Distribución de Temporadas de Series TV')
ax2.set_xlabel('Número de Temporadas')
ax2.set_ylabel('Frecuencia')
ax2.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Análisis de géneros
print("=== ANÁLISIS DE GÉNEROS ===")

# Separar géneros (algunos títulos tienen múltiples géneros)
genres_split = df_clean['listed_in'].str.split(', ').explode()
top_genres = genres_split.value_counts().head(15)

print("Top 15 géneros más populares:")
print(top_genres)

# Visualización
plt.figure(figsize=(12, 8))
top_genres.plot(kind='barh', color='purple', alpha=0.7)
plt.title('Top 15 Géneros Más Populares en Netflix', fontsize=14, fontweight='bold')
plt.xlabel('Número de Títulos')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## 5. Visualizaciones Interactivas con Plotly

In [ ]:
# Gráfico interactivo de contenido por año
yearly_interactive = df_clean.groupby(['year_added', 'type']).size().reset_index(name='count')

fig = px.line(yearly_interactive, x='year_added', y='count', color='type',
              title='Evolución del Contenido Agregado a Netflix por Año',
              labels={'year_added': 'Año', 'count': 'Número de Títulos', 'type': 'Tipo'},
              markers=True)

fig.update_layout(width=900, height=500)
fig.show()

In [ ]:
# Mapa de calor interactivo por mes y año
monthly_heatmap = df_clean.groupby(['year_added', 'month_added']).size().reset_index(name='count')
heatmap_pivot = monthly_heatmap.pivot(index='year_added', columns='month_added', values='count').fillna(0)

fig = px.imshow(heatmap_pivot, 
                title='Contenido Agregado por Mes y Año',
                labels={'x': 'Mes', 'y': 'Año', 'color': 'Títulos'},
                aspect='auto')

fig.update_layout(width=900, height=600)
fig.show()

## 6. Insights y Conclusiones

In [ ]:
print("=== INSIGHTS Y CONCLUSIONES ===")
print("\n1. DISTRIBUCIÓN DE CONTENIDO:")
movies_pct = (df_clean['type'] == 'Movie').mean() * 100
shows_pct = (df_clean['type'] == 'TV Show').mean() * 100
print(f"   - Películas: {movies_pct:.1f}%")
print(f"   - Series TV: {shows_pct:.1f}%")

print("\n2. TENDENCIAS TEMPORALES:")
peak_year = df_clean['year_added'].value_counts().index[0]
peak_count = df_clean['year_added'].value_counts().iloc[0]
print(f"   - Año pico de contenido: {peak_year} ({peak_count:,} títulos)")

recent_growth = df_clean[df_clean['year_added'] >= 2015].shape[0]
total_content = df_clean.shape[0]
recent_pct = (recent_growth / total_content) * 100
print(f"   - Contenido agregado desde 2015: {recent_pct:.1f}%")

print("\n3. GEOGRAFÍA DEL CONTENIDO:")
us_content = countries_split[countries_split == 'United States'].count()
us_pct = (us_content / len(df_clean)) * 100
print(f"   - Contenido de Estados Unidos: {us_pct:.1f}%")
print(f"   - Total de países representados: {countries_split.nunique()}")

print("\n4. CARACTERÍSTICAS DEL CONTENIDO:")
print(f"   - Duración promedio de películas: {movies_duration.mean():.0f} minutos")
print(f"   - Temporadas promedio de series: {shows_duration.mean():.1f}")
print(f"   - Géneros únicos: {genres_split.nunique()}")
print(f"   - Género más popular: {top_genres.index[0]} ({top_genres.iloc[0]:,} títulos)")

print("\n5. CLASIFICACIONES:")
most_common_rating = df_clean['rating'].value_counts().index[0]
rating_count = df_clean['rating'].value_counts().iloc[0]
rating_pct = (rating_count / len(df_clean)) * 100
print(f"   - Clasificación más común: {most_common_rating} ({rating_pct:.1f}%)")

print("\n=== RECOMENDACIONES ===")
print("1. Netflix ha experimentado un crecimiento exponencial en contenido desde 2015")
print("2. Existe una clara preferencia por películas sobre series TV")
print("3. Estados Unidos domina la producción, pero hay diversidad global")
print("4. Los dramas internacionales son el género más representado")
print("5. La plataforma mantiene un equilibrio entre diferentes clasificaciones de edad")

## 7. Exportar Dataset Limpio

In [ ]:
# Guardar el dataset limpio
output_file = 'netflix_titles_cleaned.csv'
df_clean.to_csv(output_file, index=False)
print(f"Dataset limpio guardado como: {output_file}")
print(f"Dimensiones del dataset limpio: {df_clean.shape}")

# Resumen final
print("\n=== RESUMEN DEL ANÁLISIS ===")
print(f"✅ Dataset original: {df.shape[0]:,} filas")
print(f"✅ Dataset limpio: {df_clean.shape[0]:,} filas")
print(f"✅ Filas eliminadas: {df.shape[0] - df_clean.shape[0]:,}")
print(f"✅ Columnas adicionales creadas: 5")
print(f"✅ Análisis completado exitosamente")